In [1]:
import os 
import json 
import torch
import numpy as np 
from PIL import Image
from evaluate import load
import matplotlib.pyplot as plt
from datasets import load_dataset 
from tqdm.notebook import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, AutoModelForCausalLM,  ViTImageProcessor, AutoTokenizer

In [2]:
model_configs = {
    "blip": {
        "saved_checkpoint": "checkpoints/models/blip/",
        "processor": BlipProcessor, 
        "pretrained_path": "Salesforce/blip-image-captioning-large", 
        "inference_model": BlipForConditionalGeneration
    }, 
    "git": {
        "saved_checkpoint": "checkpoints/models/git/",
        "processor": AutoProcessor, 
        "pretrained_path": "microsoft/git-base", 
        "inference_model": AutoModelForCausalLM
    },
    "vit": {
        "saved_checkpoint": "checkpoints/models/vit/",
        "processor": [ViTImageProcessor, AutoTokenizer], 
        "pretrained_path": ["nlpconnect/vit-gpt2-image-captioning", "microsoft/git-base"], 
        "inference_model": AutoModelForCausalLM
    }
}

In [3]:
DATA_PATHS = {
    "annotated_set_dir": "prompts/",
    "inference_set_dir": "complete_image_set/inference/"
}

In [4]:
def load_models(model_configs):
    models = {key: {} for key in model_configs.keys()}

    for model_name in model_configs.keys(): 
        ckpt, processor, pretrained_path, inf_model = model_configs[model_name].values()

        models[model_name]["model"] = inf_model.from_pretrained(ckpt) 
        models[model_name]["processor"] = [
            processor[0].from_pretrained(pretrained_path[0], kwargs="padding"), 
            processor[1].from_pretrained(pretrained_path[1], kwargs="padding")
        ] if model_name == "vit" else processor.from_pretrained(pretrained_path, kwargs="padding")
        
    return models

In [5]:
models = load_models(model_configs)

In [6]:
# Base configs 
def load_model_comparison(file_name): 
    image_path = DATA_PATHS["inference_set_dir"] + file_name
    inference_dict = {} 
    image_processor = None 
    
    for model_name in models.keys(): 
        model, processor = models[model_name].values()
        image = Image.open(image_path)
        
        if model_name == "vit": 
            image_processor, processor = processor
            
        pixel_values = image_processor(images=image, return_tensors="pt").pixel_values if image_processor else processor(images=image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values=pixel_values, max_length=200, num_return_sequences=5, top_k=100, do_sample=True, temperature=0.8)
        
        inference_dict[model_name] = processor.batch_decode(generated_ids, skip_special_tokens=True)

    return inference_dict

In [7]:
def print_inferences(file_name): 
    inference = load_model_comparison(file_name) 
    
    for model_name, captions in inference.items(): 
        print("Model: ", model_name.capitalize()) 
    
        for caption in captions: 
            print(caption) 
    
        print()

In [8]:
print_inferences("404.png") 

Model:  Blip
simple text based 404 page design with a white background, centered text and a white section title
a 404 page design with an elegant white and grey color scheme, centered navbar and a large screen graphic
page not found ui design with a white background, black typography, and a circle shaped illustration
create a simple about me page design with an eco themed twist with a black typography and a white background, centered with a clock and a circular logo, add a minimal section content that gives off, and links links up the content
simple white text based 404 page design with a white background and a centered title

Model:  Git
design a simple yet elegant about us page design with a natural background, geometric font that has a prominent content in dark
create an elegant landing page design for a furniture brand with ample space and centered background for content and simple text
design an elegant about me page design with minimal text and unique image
geometric text based l

In [9]:
print_inferences("about_us_test.png")

Model:  Blip
medical application landing page design with 3 sections
about us page with white and blue color scheme, 2 sections, images and text
create an elegant about us page design with a chic look and white background, rectangular sections, and a soft blue - white background
simple about us page design with white and blue scheme and yellow accents
medical application landing page design with an elegant look and white background

Model:  Git
design a simple about me page with a minimalistic and elegant look
simple about us page design with curved background and curved background, with a single section split into 2 sections, with a horizontal card layout
design a beautiful about me page with a white background, and red color scheme
design an elegant about me page with a multicolored grid, with a minimalistic, circular shape, and section arranged overlaid and another section with a white background
design a simple about me page with red and black color scheme

Model:  Vit
about us pag

In [11]:
print_inferences("about_test.jpg") 

Model:  Blip
create a landing page design with an even text - text - styled layout that has 3 sections
create an elegant landing page design for an application console for a pc application with a grey and white theme and a black and white action button
design an about us page with a white and grey theme and black and white section titles
create an elegant landing page design for an interior design brand with a grey and white theme, alternating images of interior designs with yellow accents
create an even text - only landing page design with a 2 - column layout, black and white theme, and a pop of color

Model:  Git
about us page design with horizontal stacked layout
create an elegant landing page design for a furniture brand with a white background, and black font color, with a single image column being an animated image in a horizontal composition
design a simple about me page with horizontal sections with a red, white and black color scheme
trendy about me page design for a fashion b